In [1]:
import serial
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pyvisa as visa
from pathlib import Path
from scipy.interpolate import interp1d

from saveCalData import saveMeasurement
from progressbar import update_progress
from user_prompts import yes_or_no

In [2]:
def serialQry(port, command):
    port.reset_input_buffer()
    port.write((command + '\n').encode('ascii'))
    return port.readline().decode('ascii').strip() #strip removes white space

def serialCmd(port, command):
    port.reset_input_buffer()
    port.write((command + '\n').encode('ascii'))

In [3]:
def rms(x):
    return np.sqrt(np.mean(x**2))

In [4]:
sr542 = serial.Serial(port='COM11', baudrate=115200, timeout=.5)
dc205 = serial.Serial(port='COM12', baudrate=115200, timeout=0.5)
cs580 = serial.Serial(port='COM10', baudrate=9600, timeout=0.5)
rm = visa.ResourceManager()
rm.list_resources()
dvm = rm.open_resource('USB0::0x0957::0xB318::MY52449038::INSTR')

In [5]:
print(serialQry(sr542, '*IDN?'))
print(serialQry(dc205, '*IDN?'))
print(dvm.query('*IDN?'))

Stanford_Research_Systems, SR542, s/n00001007, v0.10
Stanford_Research_Systems,DC205,s/n20500218,ver1.80
Agilent Technologies,34450A,MY52449038,00.56-00.63



In [50]:
def WaitForSpeedLock():
    serialQry(sr542, 'CHEV?') # Query the Chopper Event Register to clear it
    serialCmd(sr542, 'CHPT 4') # Look for positive transitions of the Speed Lock bit (bit 2)

    t0 = time.time()
    while True:
        time.sleep(0.5)
        event = serialQry(sr542, 'CHEV?')        
        if(event != '0'):
            break    

    print(f'Time to speed lock = {time.time() - t0:.3f} seconds')

def WaitForPhaseLock():
    serialQry(sr542, 'CHEV?') # Query the Chopper Event Register to clear it
    serialCmd(sr542, 'CHPT 8') # Look for positive transitions of the Chopper Lock bit (bit 3)

    t0 = time.time()
    while True:
        time.sleep(0.5)
        event = serialQry(sr542, 'CHEV?')        
        if(event != '0'):
            break    

    print(f'Time to phase lock = {time.time() - t0:.3f} seconds')

In [45]:
class ChopperHead():
    def __init__(self):        
        self.sn = self.GetOrSetSerialNum()
        self.quad_count = 400
        self.on_unit_tick_pos = self.GetTickPos()
        self.on_unit_curr_corr = self.GetCurrCorr()

        ticks_uncald = np.allclose(self.on_unit_tick_pos, np.linspace(0, 1, self.quad_count, endpoint=False), rtol=1e-6, atol=1e-6)
        curr_uncald = np.allclose(self.on_unit_curr_corr, np.zeros(self.quad_count), rtol=1e-6, atol=1e-6)
        if(ticks_uncald and curr_uncald):
            self.cald = False
        else:
            self.cald = True

    def GetOrSetSerialNum(self):
        serialCmd(sr542, '$CHT DNLD')
        if (serialQry(sr542, 'LERR?') == '51'):
            # Then the chopper head was in an erased state. Go ahead and upload the new s/n
            sn = str(input(f'Please enter chopper head serial number [YYWW####]').strip())
            serialCmd(sr542, f'$CSR {sn}')
            serialCmd(sr542, '$CHT UPLD')
        else:
            print(serialQry(sr542, '$CSR?'))

    def GetTickPos(self):
        tick_pos = np.linspace(0, 1, self.quad_count, endpoint=False, dtype=np.float32)
        for i in range(400):
            tick_pos[i] = np.float32(serialQry(sr542, f'$TIK? {i}'))

        return tick_pos

    def GetCurrCorr(self):
        curr_corr = np.zeros(self.quad_count, dtype=np.float32)
        for i in range(400):
            curr_corr[i] = np.float32(serialQry(sr542, f'$CCR? {i}'))

        return curr_corr

    def SpinDownMeasurement(self, start_freq_Hz, edges_to_capture='SHAFT'):
        if edges_to_capture not in ['OUTER', 'INNER', 'SHAFT']:
            raise ValueError('Invalid edges to capture')
        serialCmd(sr542, 'SRCE INT')
        serialCmd(sr542, 'CTRL SHAFT')
        serialCmd(sr542, f'IFRQ {start_freq_Hz}')

        serialCmd(sr542, 'MOTR ON')
        WaitForPhaseLock()

        serialCmd(sr542, f'$SDN {edges_to_capture}')
        time.sleep(10.0) #TODO: how long to wait here
        serialCmd(sr542, 'MOTR OFF')



In [46]:
ch_dut = ChopperHead()

21170001


In [47]:
ch_dut.cald

False